# Chapter 26: Principal Stratification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

np.random.seed(42)

%load_ext watermark
%watermark --iversions

matplotlib       : 3.7.2
pandas           : 2.0.3
matplotlib_inline: 0.1.6
statsmodels      : 0.14.0
numpy            : 1.24.3



## Partial ID

### SACE with grouped data

In [2]:
## Chapter 26.4.2
## truncation by death example
## data from Yang and Small (2016)
pi11 = 277 / (277 + 152)
pi00 = 109 / (109 + 322)
pi10 = 1 - pi11 - pi00

## observed means
mu11 = 54 / 322
mu01 = 59 / 277

## bounds on the treatment potential outcomes
lb = ((pi11 + pi10) * mu11 - pi10) / pi11
ub = ((pi11 + pi10) * mu11) / pi11
lb, ub

(0.03698057577458183, 0.19404122726930068)

In [3]:
## bounds on the sace
lb - mu01, ub - mu01

(-0.17601581411711492, -0.018955162622396077)

### SACE with microdata

In [16]:
## function for SACE with a binary outcome
def SACE01_fit(Z, M, Y):
    ## summary statistics
    pM1 = np.mean(M[Z == 1])
    pM0 = np.mean(M[Z == 0])
    mu11 = np.mean(Y[(Z == 1) & (M == 1)])
    mu01 = np.mean(Y[(Z == 0) & (M == 1)])
    ## proporitions of the strata
    pi11 = pM0
    pi00 = 1 - pM1
    pi10 = 1 - pi11 - pi00
    ## bounds on the treatment potential outcomes
    lb = ((pi11 + pi10) * mu11 - pi10) / pi11
    ub = ((pi11 + pi10) * mu11) / pi11
    ## bounds on the SACE
    return np.array([lb - mu01, ub - mu01])


def SACE01(Z, M, Y, nboot=1e3):
    from joblib import Parallel, delayed

    bounds = SACE01_fit(Z, M, Y)
    n = len(Z)

    def bootfn(*args):
        idx = np.random.choice(n, n, replace=True)
        return SACE01_fit(Z[idx], M[idx], Y[idx])

    res = Parallel(n_jobs=-1, verbose=0)(delayed(bootfn)() for _ in range(int(nboot)))
    res = np.vstack(res)
    b_se = np.std(res, axis=0)
    # IM CI
    l_ci, u_ci = bounds[0] - 1.96 * b_se[0], bounds[1] + 1.96 * b_se[1]
    res = pd.DataFrame(
        np.c_[bounds, b_se, np.array([l_ci, u_ci])],
        columns=["est", "se", "ci"],
        index=["lb", "ub"],
    )
    return res

In [17]:
## truncation by death example
## data from Yang and Small (2016)
Z = np.r_[np.repeat(1, 322 + 109), np.repeat(0, 277 + 152)]

M = np.r_[np.repeat(1, 322), np.repeat(0, 109), np.repeat(1, 277), np.repeat(0, 152)]
Y = np.r_[
    np.repeat(1, 54),
    np.repeat(0, 268),
    np.repeat(np.nan, 109),
    np.repeat(1, 59),
    np.repeat(0, 218),
    np.repeat(np.nan, 152),
]

In [19]:
yangsmall = SACE01(Z, M, Y)
yangsmall

,est,se,ci
lb,-0.176016,0.055382,-0.284565
ub,-0.018955,0.036026,0.051657


## Principal Score Methods

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning


@ignore_warnings(category=ConvergenceWarning)
def psw(Z, M, Y, X):
    pi_10 = np.mean(M[Z == 1])
    pi_00 = 1 - pi_10
    # ps_10 = sm.Logit(M[Z==1], X[Z == 1,:]).fit(disp = 0).predict(X)
    ps_10 = LogisticRegression().fit(X[Z == 1, :], M[Z == 1]).predict_proba(X)[:, 1]
    ps_00 = 1 - ps_10
    # PCEs 10 and 00
    tau_10 = (
        np.mean(Y[(Z == 1) & (M == 1)]) - np.mean(Y[Z == 0] * ps_10[Z == 0]) / pi_10
    )
    tau_00 = (
        np.mean(Y[(Z == 1) & (M == 0)]) - np.mean(Y[Z == 0] * ps_00[Z == 0]) / pi_00
    )
    return np.r_[tau_10, tau_00]


def psw_boot(Z, M, Y, X, n_boot=1e3):
    from joblib import Parallel, delayed

    point_est = psw(Z, M, Y, X)
    n = len(Z)

    def bootfn(*args):
        idx = np.random.choice(n, n, replace=True)
        return psw(Z[idx], M[idx], Y[idx], X[idx, :])

    res = Parallel(n_jobs=-1, verbose=0)(delayed(bootfn)() for _ in range(int(n_boot)))
    res = np.vstack(res)
    boot_se = np.std(res, axis=0)
    # results
    res = pd.DataFrame(
        np.c_[point_est, boot_se], columns=["est", "se"], index=["tau10", "tau00"]
    )
    return res

In [44]:
import formulaic as fm

jobsdata = pd.read_csv("jobsdata.csv")
X = (
    fm.Formula("~ sex + age + marital + nonwhite + educ + income -1")
    .get_model_matrix(data=jobsdata)
    .values
)
Z, M, Y = jobsdata[["treat", "comply", "job_seek"]].values.T
pd.crosstab(Z, M)

col_0,0.0,1.0
row_0,,
0.0,299,0
1.0,228,372


In [49]:
psw_boot(Z, M, Y, X)

,est,se
tau10,0.167204,0.101708
tau00,-0.095306,0.151533
